### Setting up the two databases
With using the cassandra database, this file will create a cassandra node with 3 keyspaces. These keyspaces are:


1. **fish_data**

which is data from Barentwatch API

2. **weather_data**
 
with data from Frost API

Here they are only created, data will be added in the following notebooks:

1. 
2. 



#### Connecting to the cassandra database

In [1]:
# Connecting to Cassandra
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

##### Setting up keyspaces for port 9042

In [2]:

session.execute(
    "CREATE KEYSPACE IF NOT EXISTS fish_data\
    WITH REPLICATION = {\
        'class' : 'SimpleStrategy',\
        'replication_factor' : 1\
    };"
)

session.execute(
    "CREATE KEYSPACE IF NOT EXISTS weather_data\
    WITH REPLICATION = {\
        'class' : 'SimpleStrategy',\
        'replication_factor' : 1\
    };"
)


#### Setting up table for more detailed information about specific locality
Inserting data from 2022

id = 35297

In [7]:
session.set_keyspace('fish_data')

#session.execute("DROP TABLE id_35297")

table_creation_query = """
    CREATE TABLE id_35297 (
        datetime TEXT PRIMARY KEY,
        avgAdultFemaleLice FLOAT,
        hasReportedLice BOOLEAN,
        avgMobileLice FLOAT,
        avgStationaryLice FLOAT,
        seaTemperature FLOAT,
    )
    """

session.execute(table_creation_query)

In [9]:
from functions import get_detailed_week_summary
import pandas as pd
from barentswatch.authentication import get_token
token = get_token()
data = []
localityid = 35297
for week in range(1, 53):
    weeksummary = get_detailed_week_summary(token, '2022', str(week), localityid)
    locality_week_data = weeksummary['localityWeek']
    datetime_object = pd.to_datetime('{}-W{}-7'.format(locality_week_data['year'],\
                                    locality_week_data['week']), format='%G-W%V-%u')
    weekly_data = {
    'datetime': datetime_object,
    'avgadultfemalelice': locality_week_data['avgAdultFemaleLice'],
    'hasreportedlice': locality_week_data['hasReportedLice'],
    'avgmobilelice': locality_week_data['avgMobileLice'],
    'avgstationarylice': locality_week_data['avgStationaryLice'],
    'seatemperature': locality_week_data['seaTemperature']
    }
    data.append(weekly_data)
df_35297 = pd.DataFrame(data)
df_35297['datetime'] = df_35297['datetime'].dt.strftime('%Y-%U-%w')

Token request successful
(52, 6)


,datetime,avgadultfemalelice,hasreportedlice,avgmobilelice,avgstationarylice,seatemperature
0,2022-02-0,0.05,True,0.14,0.05,6.1
1,2022-03-0,0.06,True,0.44,0.04,5.5
2,2022-04-0,0.09,True,0.35,0.01,6.9


#### Creating table for all fish localitties and inserting data for 2022 into it

- Include all variables from Barentwatch 

In [ ]:

table_creation_query = """
    CREATE TABLE IF NOT EXISTS locality_data (
        year INT,
        week INT,
        localityNo INT,
        localityWeekId INT PRIMARY KEY,
        name TEXT,
        hasReportedLice BOOLEAN,
        isFallow BOOLEAN,
        avgAdultFemaleLice DOUBLE,
        hasCleanerfishDeployed BOOLEAN,
        hasMechanicalRemoval BOOLEAN,
        hasSubstanceTreatments BOOLEAN,
        hasPd BOOLEAN,
        hasIla BOOLEAN,
        municipalityNo TEXT,
        municipality TEXT,
        lat DOUBLE,
        lon DOUBLE,
        isOnLand BOOLEAN,
        inFilteredSelection BOOLEAN,
        hasSalmonoids BOOLEAN,
        isSlaughterHoldingCage BOOLEAN
    );
"""
session.execute(table_creation_query) # Uncomment to create the table

# Define the INSERT statement
insert_query = """
    INSERT INTO locality_data (year, week, localityNo, localityWeekId, name, hasReportedLice, isFallow, 
                                avgAdultFemaleLice, hasCleanerfishDeployed, hasMechanicalRemoval, 
                                hasSubstanceTreatments, hasPd, hasIla, municipalityNo, municipality, 
                                lat, lon, isOnLand, inFilteredSelection, hasSalmonoids, isSlaughterHoldingCage)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
"""

In [ ]:

for week in range(1, 53):
    print(f'Inserting data for week {week}...')
    weeksummary = get_week_summary(token, '2021', str(week))

    for locality in weeksummary['localities']:
        session.execute(
            insert_query,
            (
                weeksummary['year'],
                weeksummary['week'],
                locality['localityNo'],
                locality['localityWeekId'],
                locality['name'],
                locality['hasReportedLice'],
                locality['isFallow'],
                locality['avgAdultFemaleLice'],
                locality['hasCleanerfishDeployed'],
                locality['hasMechanicalRemoval'],
                locality['hasSubstanceTreatments'],
                locality['hasPd'],
                locality['hasIla'],
                locality['municipalityNo'],
                locality['municipality'],
                locality['lat'],
                locality['lon'],
                locality['isOnLand'],
                locality['inFilteredSelection'],
                locality['hasSalmonoids'],
                locality['isSlaughterHoldingCage']
            )
        )